# Get HydroVu data

In [5]:
import httplib2
import requests
import json
import pandas
import datetime
import pytz
import urllib
import time

from config import locations_bowen,locations_hammond,locations_wansley

In [ ]:
def get_token(client_id,client_secret):
    
    token_url = 'https://www.hydrovu.com/public-api/oauth/token'
    
    
    data = {'grant_type': 'client_credentials'}
    #print("requesting access token")
    access_token_response = requests.post(token_url, data=data, verify=False, allow_redirects=False, auth=(client_id, client_secret))

    #print("response")
    #print(access_token_response.headers)
    #print('body: ' + access_token_response.text)

    # we can now use the access_token as much as we want to access protected resources.
    tokens = json.loads(access_token_response.text)
    access_token = tokens['access_token']
    print("access token: " + access_token)

    return access_token

In [ ]:


def get_data(token,nextpage=None,loc_id=None,startdate=None):

    if not loc_id:
        #Get valid values
        uri     = 'https://www.hydrovu.com/public-api/v1/sispec/friendlynames'
    
        uri_all = uri
    
    else:
        #Get Data
        uri     = "https://www.hydrovu.com/public-api/v1/locations/" + loc_id + "/data?"
    
        payload = {'startTime':startdate}
    
        body    = urllib.parse.urlencode(payload)
    
        uri_all = uri + body

    print(uri_all)
    #location_req = "https://www.hydrovu.com/public-api/v1/locations/list"
    
    #Get the data
    req_method = 'GET'
    
    if not nextpage:
        req_headers   = {'Content-Type':'application/json','Authorization': 'Bearer ' + token}
    else:
        req_headers   = {'Content-Type':'application/json','Authorization':'Bearer ' + token,'X-ISI-Start-Page':nextpage}
    
    
    http              = httplib2.Http() 
    http              = httplib2.Http(disable_ssl_certificate_validation=True) #to disable Cato cert issues
    
    #use a while loop in case the API fails
    status = 'na'
    tries = 0
    response = {}
    data = {}
    
    #Clean this up later
    while status != '200':

        try:
            response, content = http.request(uri_all, req_method, headers=req_headers)
            status = response['status']
            data = json.loads(content)
            #print('Success')

        except:
            print('Failed')
        
        #increment the tries
        tries += 1
        #print(tries)
        
        #exit the loop after too many tries.
        if tries > 5:
            status = '200'
            print("Download failed.  Status: {} after {} tries.".format(response['status'], tries))
        pass
    
    return response, data

In [ ]:
#Get API Token
api='a93615fda9264cd587e70d33f6881479'
client='Geosyntec'

token = get_token(client,api)


# Get Valid Values

In [ ]:
#Get Friendly Data
res, friend_data = get_data(token)

In [ ]:
friend_data

In [ ]:
param_df = pandas.json_normalize(friend_data['parameters']).transpose().reset_index()
param_df.columns = ['parameterId','parameter']
param_df#.info()

In [ ]:
unit_df = pandas.json_normalize(friend_data['units']).transpose().reset_index()
unit_df.columns = ['unitId','unit']
unit_df.head()

# Get Data

In [ ]:
#Which locations to look at
locations_all = locations_bowen + locations_wansley + locations_hammond
#locations_all

In [ ]:
ts_epoch = round(time.time()) - 14400
ts = datetime.datetime.fromtimestamp(ts_epoch).strftime('%Y%m%d_%H%M')
ts

In [ ]:
#manual start time
#Time is in Unix Epoch seconds (seconds since 1/1/1970) 🥴 
#For help https://www.epochconverter.com/  1641402000
start = round(time.time()) - 121600
#start = 1641579447

In [ ]:
#Gets all data
#start=0

In [ ]:
point_data=[]

for x in locations_hammond:
 
    locName = x['name']
    
    for y in x['id']:
        print(y)
        locId = str(y)

        resp, first_data = get_data(token,None,locId,start)
        
        if resp['status'] == '200':
            point_data.append(first_data)
        
        while 'x-isi-next-page' in resp:

            print(resp['x-isi-next-page'])
            resp, content_next = get_data(token,resp['x-isi-next-page'],locId,start)
            
            if resp['status'] == '200':
                point_data.append(content_next)

            pass

In [ ]:
#resp

In [3]:
response = {
  "TableData": {
    "Columns": [
      {
        "DataTypeUniqueName": "GMS600.InputVoltage",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "Voltage"
      },
      {
        "DataTypeUniqueName": "GMS600.Temperature",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "Temperature"
      },
      {
        "DataTypeUniqueName": "GMS600.FixQuality",
        "DataTypeClassName": "System.Int32",
        "UnitGroupName": "RawInteger"
      },
      {
        "DataTypeUniqueName": "GMS600.ConvergenceTime",
        "DataTypeClassName": "System.Int32",
        "UnitGroupName": "RawInteger"
      },
      {
        "DataTypeUniqueName": "GMS600.CorrectionPackets",
        "DataTypeClassName": "System.Int32",
        "UnitGroupName": "RawInteger"
      },
      {
        "DataTypeUniqueName": "GMS600.BaselineLength",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "Distance"
      },
      {
        "DataTypeUniqueName": "GMS600.Latitude",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "LatLon"
      },
      {
        "DataTypeUniqueName": "GMS600.Longitude",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "LatLon"
      },
      {
        "DataTypeUniqueName": "GMS600.Height",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "LocalCoordinate"
      },
      {
        "DataTypeUniqueName": "GMS600.AntennaHeight",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "Distance"
      },
      {
        "DataTypeUniqueName": "GMS600.Northing",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "LocalCoordinate"
      },
      {
        "DataTypeUniqueName": "GMS600.Easting",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "LocalCoordinate"
      },
      {
        "DataTypeUniqueName": "GMS600.Elevation",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "LocalCoordinate"
      },
      {
        "DataTypeUniqueName": "GMS600.DeltaNorth",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "Displacement"
      },
      {
        "DataTypeUniqueName": "GMS600.DeltaEast",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "Displacement"
      },
      {
        "DataTypeUniqueName": "GMS600.DeltaElevation",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "Displacement"
      },
      {
        "DataTypeUniqueName": "GMS600.Displacement_Hz",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "Displacement"
      },
      {
        "DataTypeUniqueName": "GMS600.Displacement_Vt",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "Displacement"
      },
      {
        "DataTypeUniqueName": "GMS600.Displacement_3D",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "Displacement"
      },
      {
        "DataTypeUniqueName": "GMS600.Displacement_Avg_Hz",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "Displacement"
      },
      {
        "DataTypeUniqueName": "GMS600.Displacement_Avg_Vt",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "Displacement"
      },
      {
        "DataTypeUniqueName": "GMS600.AverageDisplacement_3D",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "Displacement"
      },
      {
        "DataTypeUniqueName": "GMS600.Velocity_3D_Point1",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "Velocity"
      },
      {
        "DataTypeUniqueName": "GMS600.InverseVelocity_3D_Point1",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "InverseVelocity"
      },
      {
        "DataTypeUniqueName": "GMS600.Velocity_3D_Point2",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "Velocity"
      },
      {
        "DataTypeUniqueName": "GMS600.InverseVelocity_3D_Point2",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "InverseVelocity"
      },
      {
        "DataTypeUniqueName": "GMS600.Velocity_3D_Period1",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "Velocity"
      },
      {
        "DataTypeUniqueName": "GMS600.InverseVelocity_3D_Period1",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "InverseVelocity"
      },
      {
        "DataTypeUniqueName": "GMS600.Velocity_3D_Period2",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "Velocity"
      },
      {
        "DataTypeUniqueName": "GMS600.InverseVelocity_3D_Period2",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "InverseVelocity"
      },
      {
        "DataTypeUniqueName": "Transit.Azimuth",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "Angles"
      },
      {
        "DataTypeUniqueName": "Transit.Plunge",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "Angles"
      },
      {
        "DataTypeUniqueName": "GMS600.TiltA",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "Tilt"
      },
      {
        "DataTypeUniqueName": "GMS600.TiltB",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "Tilt"
      },
      {
        "DataTypeUniqueName": "GMS600.DeltaTiltA",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "Tilt"
      },
      {
        "DataTypeUniqueName": "GMS600.DeltaTiltB",
        "DataTypeClassName": "System.Double",
        "UnitGroupName": "Tilt"
      },
      {
        "DataTypeUniqueName": "GMS600.MagX",
        "DataTypeClassName": "System.Int32",
        "UnitGroupName": "RawInteger"
      },
      {
        "DataTypeUniqueName": "GMS600.MagY",
        "DataTypeClassName": "System.Int32",
        "UnitGroupName": "RawInteger"
      },
      {
        "DataTypeUniqueName": "GMS600.MagZ",
        "DataTypeClassName": "System.Int32",
        "UnitGroupName": "RawInteger"
      }
    ],
    "Rows": [
      {
        "Epoch": "2022-01-31T17:15:09",
        "IsDeleted": False,
        "Values": [
          13.24,
          5.6,
          "null",
          43,
          80,
          "null",
          0.6845671370364073,
          -1.433710137598036,
          242.75790999999998,
          0,
          5229111.682240199,
          4065368.2997571146,
          242.75790999999998,
          0.016449828632175922,
          0.000851459801197052,
          -0.03008000000005495,
          0.01647185010316113,
          -0.03008000000005495,
          0.03429472621007952,
          "null",
          "null",
          "null",
          "null",
          "null",
          "null",
          "null",
          "null",
          "null",
          "null",
          "null",
          0.05171486154547635,
          -1.0697965165946908,
          -21.43275218771322,
          -23.063418206736348,
          4.123049823606433,
          -0.582431249805726,
          "null",
          "null",
          "null"
        ]
      },
      {
        "Epoch": "2022-01-31T18:15:11",
        "IsDeleted": False,
        "Values": [
          13.27,
          8.74,
          "null",
          65,
          117,
          "null",
          0.6845671369278825,
          -1.4337101368706877,
          242.75678,
          0,
          5229111.683545234,
          4065368.3038066365,
          242.75678,
          0.017754863016307354,
          0.004900981672108173,
          -0.031210000000044147,
          0.01841887027149545,
          -0.031210000000044147,
          0.03623974174964461,
          "null",
          "null",
          "null",
          "null",
          "null",
          "null",
          "null",
          0.0000012221315696194557,
          818242.5074833616,
          "null",
          "null",
          0.2693291324550693,
          -1.0376440937549976,
          -21.12752063727652,
          -23.20993992036654,
          4.428281374043134,
          -0.7289529634359191,
          "null",
          "null",
          "null"
        ]
      }
    ]
  },
  "Exception": "null"
}

In [4]:
response

{'TableData': {'Columns': [{'DataTypeUniqueName': 'GMS600.InputVoltage',
    'DataTypeClassName': 'System.Double',
    'UnitGroupName': 'Voltage'},
   {'DataTypeUniqueName': 'GMS600.Temperature',
    'DataTypeClassName': 'System.Double',
    'UnitGroupName': 'Temperature'},
   {'DataTypeUniqueName': 'GMS600.FixQuality',
    'DataTypeClassName': 'System.Int32',
    'UnitGroupName': 'RawInteger'},
   {'DataTypeUniqueName': 'GMS600.ConvergenceTime',
    'DataTypeClassName': 'System.Int32',
    'UnitGroupName': 'RawInteger'},
   {'DataTypeUniqueName': 'GMS600.CorrectionPackets',
    'DataTypeClassName': 'System.Int32',
    'UnitGroupName': 'RawInteger'},
   {'DataTypeUniqueName': 'GMS600.BaselineLength',
    'DataTypeClassName': 'System.Double',
    'UnitGroupName': 'Distance'},
   {'DataTypeUniqueName': 'GMS600.Latitude',
    'DataTypeClassName': 'System.Double',
    'UnitGroupName': 'LatLon'},
   {'DataTypeUniqueName': 'GMS600.Longitude',
    'DataTypeClassName': 'System.Double',
    'Un

In [7]:
df = pandas.json_normalize(response)

In [8]:
df

,Exception,TableData.Columns,TableData.Rows
0,null,"[{'DataTypeUniqueName': 'GMS600.InputVoltage',...","[{'Epoch': '2022-01-31T17:15:09', 'IsDeleted':..."


In [ ]:
data_df_all = pandas.DataFrame(columns=['timestamp','value','parameterId','unitId','location','locationID'])

for x in point_data:
    data_df = pandas.json_normalize(x['parameters'], ['readings'], ['parameterId','unitId'])
    data_df['location'] = locName
    data_df['locationID'] = x['locationId']
    data_df_all = data_df_all.append(data_df)
    
data_df_all

In [ ]:
new_df = pandas.merge(data_df_all, param_df, on='parameterId')
new_df = pandas.merge(new_df, unit_df, on='unitId')
new_df['timestamp'] = pandas.to_datetime(new_df['timestamp'],unit='s')
new_df['timestamp'] = new_df['timestamp'].dt.tz_localize('utc').dt.tz_convert('US/Eastern').dt.tz_localize(None)
new_df.value = new_df.value.round(3)
new_df

In [ ]:
#This is specifically for Wansley
vulinks = [6460467848675328]

for i, row in new_df.iterrows():
    #print(row.locationID)
    
    if row.locationID in vulinks and row.parameter == 'Temperature':
        print(row.locationID)
        new_df.at[i,'parameter'] = 'Temperature_AMB'
        
new_df

In [ ]:
duplicateRowsDF = new_df[new_df.duplicated(['timestamp','parameterId'])]
duplicateRowsDF.to_csv('dups.csv')
duplicateRowsDF

In [ ]:
#Added temp amb and level for Wansley

if 'PZ' in locName:
    cols=['Battery Level','Baro','Temperature','Depth','Pressure','Level: Elevation','Temperature_AMB']
else:
    cols=['Battery Level','Baro','Temperature','% Saturation O₂','DO','Depth','ORP','Pressure','Specific Conductivity','pH']



crosstab_data = new_df.pivot(index='timestamp',columns='parameter', values='value').reindex(columns=cols)
crosstab_data

In [ ]:
crosstab_data.to_csv(filename)

# Extra Stuff


In [ ]:
#resp

In [ ]:
all_data = []

In [ ]:
all_data = []
data_df_everything = pandas.DataFrame(columns=['timestamp','value','parameterId','unitId','location','locationID'])
for x in locations_all:
    
    point_data=[]
 
    locName = x['name']
    
    data_df_all = pandas.DataFrame(columns=['timestamp','value','parameterId','unitId','location','locationID'])
    
    for y in x['id']:
        print(y)
        locId = str(y)

        resp, first_data = get_data(token,None,locId,start)
        
        if resp['status'] == '200':
            point_data.append(first_data)
        
        while 'x-isi-next-page' in resp:

            #print(resp['x-isi-next-page'])
            resp, content_next = get_data(token,resp['x-isi-next-page'],locId,start)
            
            if resp['status'] == '200':
                point_data.append(content_next)

            pass


    all_data.append(point_data)

        #Put all data into 1 dataframe
    for x in point_data:
        #print(x)
        data_df = pandas.json_normalize(x['parameters'], ['readings'], ['parameterId','unitId'])
        data_df['location'] = locName
        data_df['locationID'] = x['locationId']
        data_df_all = data_df_all.append(data_df)

    #Add valid values, convert UNIX time to eastern timezone, clean up decimals
    new_df = pandas.merge(data_df_all, param_df, on='parameterId')
    new_df = pandas.merge(new_df, unit_df, on='unitId')
    new_df['timestamp'] = pandas.to_datetime(new_df['timestamp'],unit='s')
    new_df['timestamp'] = new_df['timestamp'].dt.tz_localize('utc').dt.tz_convert('US/Eastern').dt.tz_localize(None)
    new_df.value = new_df.value.round(3)
    
    
    
    #This is specifically for Wansley - Vulinks that are reporting ambient temperature
    vulinks = [6460467848675328,5554726447087616,6684526989541376]

    for i, row in new_df.iterrows():
        #print(row.locationID)
        
        if row.locationID in vulinks and row.parameter == 'Temperature':
            #print(row.locationID)
            new_df.at[i,'parameter'] = 'Temperature_AMB'

    data_df_everything = data_df_everything.append(new_df)        
    
    #pivot the row data into column data for iSite then output to timestamped file
    if 'PZ' in locName:
        cols=['Battery Level','Baro','Temperature','Depth','Pressure']
    else:
        cols=['Battery Level','Baro','Temperature','% Saturation O₂','DO','Depth','ORP','Pressure','Specific Conductivity','pH']

    crosstab_data = new_df.pivot(index='timestamp',columns='parameter', values='value').reindex(columns=cols)

    #filename = ts + '_' + locName + '.dat'
    #crosstab_data.to_csv(filename)

In [ ]:
data_df_everything.to_csv('parameters.csv')

In [ ]:
data_df_all = pandas.DataFrame(columns=['timestamp','value','parameterId','unitId','location','locationID'])
for x in all_data:
    print(x)
    data_df = pandas.DataFrame(columns=['timestamp','value','parameterId','unitId','location','locationID'])
    data_df = pandas.json_normalize(x['parameters'], ['readings'], ['parameterId','unitId'])
    #data_df['location'] = locName
    data_df['locationID'] = x['locationId']
    data_df_all = data_df_all.append(data_df)

In [ ]:
data_df_all

In [ ]:
all_data

In [ ]:
for x in all_data[:2]:
    print(x)

In [ ]:
data_df_all

In [ ]:
duplicateRowsDF = new_df[new_df.duplicated(['timestamp','parameterId'])]
#duplicateRowsDF.to_csv('dups.csv')
duplicateRowsDF

In [ ]:
point_data

In [ ]:
content_next